In [ ]:
!pip install -U albumentations


In [ ]:
import albumentations as A
print("Albumentations version:", A.__version__)


Albumentations version: 1.4.20


In [ ]:
!rm albumentations.py


rm: cannot remove 'albumentations.py': No such file or directory


In [ ]:
# نصب کتابخانه‌های مورد نیاز
!pip install gdown
!pip install tqdm
!pip install huggingface_hub
!pip install timm
!pip install torchmetrics
!pip install albumentations
!pip install torchsummary
!pip install scikit-learn
!pip install pytorch-lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 45.5 MB/s eta 0:00:00


In [ ]:
import os
from huggingface_hub import hf_hub_download
import tarfile

# دانلود دیتاست
hf_hub_download(repo_id='RayanAi/inat_train_modified',
               filename="inat_train_modified.tar.gz",
               repo_type="dataset",
               local_dir=".")

# استخراج فایل tar.gz
with tarfile.open("inat_train_modified.tar.gz", "r:gz") as tar:
    tar.extractall(path=".")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


inat_train_modified.tar.gz:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

In [ ]:
from typing import Iterator
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class Node:
    def __init__(self, name):
        self.name = name
        self._count = 0
        self.children = {}
        self._entities = []

    def add_to_node(self, path, entity, level=0):
        if level >= len(path):
            self._entities.append(entity)
            return
        part = path[level]
        if part not in self.children:
            self.children[part] = Node(path[:level+1])
        self.children[part].add_to_node(path, entity, level=level+1)
        self._count += 1

    @property
    def is_leaf(self):
        return len(self._entities) > 0

    @property
    def count(self):
        if self.is_leaf:
            return len(self._entities)
        else:
            return self._count

    @property
    def entities(self):
        if self.is_leaf:
            return list((entity, self.name) for entity in self._entities)
        else:
            child_entities = []
            for child in self.children.values():
                child_entities.extend(child.entities)
        return child_entities

    def level_iterator(self, level=None):
        if level == 0:
            yield self
        elif level is None and self.is_leaf:
            yield self
        elif self.is_leaf and level != 0:
            raise Exception("Incorrect level is specified in tree.")
        else:
            if level is not None:
                level -= 1
            for child in self.children.values():
                for v in child.level_iterator(level):
                    yield v

    def print_node(self, level=0, max_level=None):
        print(' ' * (level * 4) + f"{self.name[-1]} ({self.count})")
        for node in self.children.values():
            if max_level is None or level < max_level:
                node.print_node(level + 1, max_level=max_level)
        return

class HierarchicalDataset(Dataset):
    def __init__(self, dataset_path, level=None, transform=None, verbose=True):
        self.tree = Node(("Dataset",))  # Initialize with root
        self.level = level if level is not None else 2  # تغییر به سطح 2 بر اساس قوانین مسابقه
        self.classes = set()
        self.data = []
        self.transform = transform

        index = 0
        for group_name in sorted(os.listdir(dataset_path)):
            group_dir = os.path.join(dataset_path, group_name)
            if not os.path.isdir(group_dir):
                continue
            for image_name in sorted(os.listdir(group_dir)):
                image_path = os.path.join(group_dir, image_name)
                group = tuple(group_name.split("_")[1:])  # Assuming format like 'class_name'
                if len(group) < self.level:
                    continue  # Skip if group path is shorter than required level
                group = group[:self.level]
                self.data.append({
                    "image_path": image_path,
                    "group": group,
                })
                self.tree.add_to_node(group, index)
                index += 1
                self.classes.add(group)

        self.classes = {group: idx for idx, group in enumerate(sorted(list(self.classes)))}

        if verbose:
            print(f"Dataset Length: {len(self.data)}")
            print("Hierarchical Structure (up to level 2):")
            self.tree.print_node(max_level=2)
            print(f"Number of classes: {len(self.classes)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        image = Image.open(sample["image_path"]).convert('RGB')
        group = sample["group"][:self.level]
        target = self.classes[group]
        if self.transform:
            image = self.transform(image)
        return image, target

    def get_group_iterator(self, level=None) -> Iterator[Node]:
        for group in self.tree.level_iterator(level):
            yield group


In [ ]:
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch

# تعریف MixUp
def mixup_data(x, y, alpha=0.2):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if torch.cuda.is_available():
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# تعریف CutMix
def cutmix_data(x, y, alpha=1.0):
    '''Returns cutmixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size, C, H, W = x.size()
    if torch.cuda.is_available():
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    # تعریف منطقه برای برش
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    rw = int(W * np.sqrt(1 - lam))
    rh = int(H * np.sqrt(1 - lam))

    x1 = np.clip(cx - rw // 2, 0, W)
    y1 = np.clip(cy - rh // 2, 0, H)
    x2 = np.clip(cx + rw // 2, 0, W)
    y2 = np.clip(cy + rh // 2, 0, H)

    x[:, :, y1:y2, x1:x2] = x[index, :, y1:y2, x1:x2]
    lam = 1 - ((x2 - x1) * (y2 - y1) / (W * H))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

# تعریف داده‌افزایی برای آموزش با استفاده از Albumentations
train_transform = A.Compose([
    # ترکیب چندین تکنیک داده‌افزایی
    A.OneOf([
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        A.MotionBlur(blur_limit=5, p=0.3),
    ], p=1.0),
    A.Resize(256, 256),
    A.RandomResizedCrop(224, 224, scale=(0.8, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=(0.4556, 0.4714, 0.3700), std=(0.2370, 0.2318, 0.2431)),
    A.RandomErasing(p=0.1),
    ToTensorV2()
])

# پیش‌پردازش برای مجموعه اعتبارسنجی
val_transform = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(224, 224),
    A.Normalize(mean=(0.4556, 0.4714, 0.3700), std=(0.2370, 0.2318, 0.2431)),
    ToTensorV2()
])


AttributeError: module 'albumentations' has no attribute 'RandomErasing'

In [ ]:
import torch
from torchvision import models
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
from torch.amp import GradScaler  # استفاده از torch.amp
from torchmetrics import Accuracy
from sklearn.model_selection import StratifiedKFold
import timm  # برای مدل‌های پیشرفته‌تر

def get_model(model_name, num_classes):
    model = timm.create_model(model_name, pretrained=True)
    in_features = model.get_classifier().in_features
    model.classifier = nn.Linear(in_features, num_classes)
    return model

# تنظیم دستگاه
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# بارگذاری دیتاست
dataset_path = 'train'  # مسیر دیتاست
full_dataset = HierarchicalDataset(dataset_path=dataset_path, level=2, transform=train_transform)

# محاسبه وزن‌های کلاس
def compute_class_weights(dataset):
    class_counts = {}
    for _, label in dataset:
        class_counts[label] = class_counts.get(label, 0) + 1
    total_samples = len(dataset)
    num_classes = len(full_dataset.classes)
    class_weights = [total_samples / (num_classes * class_counts[i]) for i in range(num_classes)]
    return torch.tensor(class_weights, dtype=torch.float).to(device)

class_weights = compute_class_weights(full_dataset)

# تعریف Sampler گروه‌محور
from torch.utils.data import Sampler
import math

class GroupBalancedSampler(Sampler):
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.groups = {}
        for idx in range(len(self.dataset)):
            _, label = self.dataset[idx]
            if label not in self.groups:
                self.groups[label] = []
            self.groups[label].append(idx)
        self.group_keys = list(self.groups.keys())
        self.num_groups = len(self.group_keys)
        self.samples_per_group = math.ceil(batch_size / self.num_groups)

    def __iter__(self):
        indices = []
        for group in self.group_keys:
            group_indices = self.groups[group]
            if len(group_indices) < self.samples_per_group:
                group_indices = group_indices * (self.samples_per_group // len(group_indices) + 1)
            selected = group_indices[:self.samples_per_group]
            indices.extend(selected)
        np.random.shuffle(indices)
        return iter(indices)

    def __len__(self):
        return self.batch_size * math.ceil(len(self.dataset) / self.batch_size)

# تعریف مدل‌های مختلف برای Ensemble
model_names = ['efficientnet_b4', 'resnest50d', 'resnet50']
models_list = [get_model(name, num_classes=len(full_dataset.classes)).to(device) for name in model_names]

# تعریف Ensemble مدل‌ها
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)
        for model in self.models:
            for param in model.parameters():
                param.requires_grad = False  # Freeze all models
        self.fc = nn.Linear(len(models), len(full_dataset.classes))

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        outputs = torch.stack(outputs, dim=1)  # Shape: (batch_size, num_models, num_classes)
        outputs = outputs.mean(dim=1)  # Averaging outputs
        return outputs

ensemble_model = EnsembleModel(models_list).to(device)

# تعریف Focal Loss با استفاده از وزن‌های کلاس و Label Smoothing
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, smoothing=0.1, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smoothing = smoothing
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Apply label smoothing
        num_classes = inputs.size(1)
        with torch.no_grad():
            true_dist = torch.zeros_like(inputs)
            true_dist.fill_(self.smoothing / (num_classes - 1))
            true_dist.scatter_(1, targets.data.unsqueeze(1), 1.0 - self.smoothing)

        ce_loss = -true_dist * torch.log_softmax(inputs, dim=1)
        if self.alpha is not None:
            ce_loss = ce_loss * self.alpha.unsqueeze(0)

        ce_loss = ce_loss.sum(dim=1)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

criterion = FocalLoss(alpha=class_weights, gamma=2, smoothing=0.1, reduction='mean')

# تعریف بهینه‌ساز
optimizer = optim.AdamW(ensemble_model.fc.parameters(), lr=1e-3, weight_decay=1e-5)  # Only train the final layer

# تعریف تنظیم‌کننده نرخ یادگیری پیشرفته‌تر
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# تعریف GradScaler برای mixed precision
scaler = GradScaler()

# تقسیم‌بندی به مجموعه آموزشی و اعتبارسنجی با StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(full_dataset)), [label for _, label in full_dataset])):
    print(f"Fold {fold + 1}")
    train_subset = torch.utils.data.Subset(full_dataset, train_idx)
    val_subset = torch.utils.data.Subset(full_dataset, val_idx)

    # بارگذاری داده‌ها با استفاده از GroupBalancedSampler
    train_sampler = GroupBalancedSampler(train_subset, batch_size=64)
    train_loader = DataLoader(train_subset, batch_size=64, sampler=train_sampler, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

    # تعریف Early Stopping
    best_val_acc = 0.0
    early_stopping_patience = 7
    early_stopping_counter = 0

    for epoch in range(1, 51):  # افزایش تعداد Epoch‌ها
        print(f"Epoch {epoch}/50")

        # آموزش
        ensemble_model.train()
        running_loss = 0.0
        accuracy_metric = Accuracy(task="multiclass", num_classes=len(full_dataset.classes)).to(device)
        for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            # اعمال MixUp و CutMix
            mixed_inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, alpha=0.2)
            mixed_inputs, targets_a, targets_b, lam = cutmix_data(mixed_inputs, targets_a, alpha=1.0)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():  # استفاده از autocast برای mixed precision
                outputs = ensemble_model(mixed_inputs)
                loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            scheduler.step(epoch + epoch / len(train_loader))

            running_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            accuracy_metric.update(preds, labels)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = accuracy_metric.compute().item()
        print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.2f}%")

        # اعتبارسنجی
        ensemble_model.eval()
        running_loss = 0.0
        accuracy_metric = Accuracy(task="multiclass", num_classes=len(full_dataset.classes)).to(device)
        group_correct = {}
        group_total = {}
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation", leave=False):
                inputs, labels = inputs.to(device), labels.to(device)

                with torch.cuda.amp.autocast():
                    outputs = ensemble_model(inputs)
                    loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                preds = torch.argmax(outputs, dim=1)
                accuracy_metric.update(preds, labels)

                # به‌روزرسانی دقت هر گروه
                for i in range(len(labels)):
                    label = labels[i].item()
                    pred = preds[i].item()
                    if label not in group_correct:
                        group_correct[label] = 0
                        group_total[label] = 0
                    if pred == label:
                        group_correct[label] += 1
                    group_total[label] += 1

        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_acc = accuracy_metric.compute().item()

        # محاسبه دقت بر اساس K پایین‌ترین گروه‌ها (فرض K=10%)
        def calculate_top_k_group_accuracy(group_correct, group_total, K=10):
            # محاسبه دقت هر گروه
            group_acc = {label: group_correct.get(label, 0) / group_total.get(label, 1) for label in group_total}
            # مرتب‌سازی گروه‌ها بر اساس دقت به ترتیب صعودی
            sorted_groups = sorted(group_acc.items(), key=lambda x: x[1])
            # انتخاب K پایین‌ترین گروه
            top_k_groups = sorted_groups[:K]
            # محاسبه میانگین دقت K گروه
            top_k_avg_acc = sum([acc for _, acc in top_k_groups]) / K
            return top_k_avg_acc

        K = max(1, int(0.1 * len(group_correct)))  # K=10%
        top_k_avg_acc = calculate_top_k_group_accuracy(group_correct, group_total, K=K)

        print(f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}% | Top K Acc: {top_k_avg_acc:.2f}%")

        # ذخیره چک‌پوینت بهترین مدل بر اساس Top K Accuracy
        if top_k_avg_acc > best_val_acc:
            best_val_acc = top_k_avg_acc
            checkpoint_path = os.path.join(checkpoint_dir, f'best_checkpoint_fold{fold + 1}.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': ensemble_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': epoch_val_loss,
                'val_acc': top_k_avg_acc,
            }, checkpoint_path)
            print(f"New best model saved with Top K Acc: {best_val_acc:.2f}%")
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break

        print("-" * 30)

    print(f"Best Val Top K Acc for Fold {fold + 1}: {best_val_acc:.2f}%")


In [ ]:
# تعریف TTA
def get_test_transforms():
    return A.Compose([
        A.Resize(256, 256),
        A.CenterCrop(224, 224),
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        A.Rotate(limit=15, p=0.5),
        A.Normalize(mean=(0.4556, 0.4714, 0.3700), std=(0.2370, 0.2318, 0.2431)),
        ToTensorV2()
    ])

# تعریف دیتاست و DataLoader برای تست
test_dataset = HierarchicalDataset(dataset_path='test', level=2, transform=get_test_transforms(), verbose=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

# تعریف مدل Ensemble
ensemble_model.eval()
predictions = []
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Testing", leave=False):
        inputs = inputs.to(device)
        outputs = ensemble_model(inputs)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())

# ذخیره نتایج
import pandas as pd

# فرض کنید شما یک فایل نمونه‌ی Submission.csv دارید که شامل شناسه‌های تصاویر است
submission = pd.read_csv('sample_submission.csv')
submission['label'] = predictions
submission.to_csv('submission.csv', index=False)

print("پیش‌بینی‌ها ذخیره شد در 'submission.csv'")
